In [3]:
import torch
import os
import pandas
import pickle
import time

sentence_folder_path = "data/sentences_new"
processed_data_folder_path = "processed_data"

In [4]:
def convert_to_binary_sentence_data(data_folder_path,processed_data_folder_path,
                                    data_binary_name="data", data_dump_limit=100000,
                                    data_count_limit = 99999999,verbose=1):
    # data count limit is how many txt files do you want in total
    # data dump limit is every how many txt files do you want to do a binary dump
    start_time = time.time()
    # for test purposes, data limit can be set to indicate how much data to use
    data_count = 0
    # give the circuit court main folder's path, read all data
    folder_names = os.listdir(data_folder_path)
    
#   judge_df = pandas.DataFrame(columns=["Judge_Name","Year","Sentence"])
    data_list = []
    save_part_index = 0
    
    for folder_name in folder_names: # for each folder
        if verbose > 0:
            print("now process:",folder_name,"current data count:",data_count,"time used:",time.time()-start_time)
        
        year = folder_name[-4:]
        data_file_names = os.listdir(os.path.join(data_folder_path,folder_name))
        for file_name in data_file_names: # for each file
            file_name_tokens = file_name.split(".")
            file_name_tokens = file_name_tokens[0].split("_")
            judge_name = file_name_tokens[2] # we get the judge's name from the file name
            
            file_path = os.path.join(data_folder_path,folder_name,file_name)
            fpt = open(file_path,"r")
            sentence = fpt.read()
            fpt.close()
            
            new_data_entry = [year,judge_name,sentence]
            data_list.append(new_data_entry)
            
            data_count += 1
            
            if data_count>10 and (data_count+1)%data_dump_limit==0:
                print("dumped part",save_part_index,"time used:",time.time()-start_time)
                pickle.dump(data_list, open(os.path.join(processed_data_folder_path,
                                                    data_binary_name+str(save_part_index)), "wb"))
                save_part_index += 1
                data_list = []
            if data_count > data_count_limit: # for debugging purposes
                return data_count
    
    if data_count>10 and not (data_count+1)%data_dump_limit==0: # if have final part then dump final part
                print("dumped part",save_part_index,"time used:",time.time()-start_time)
                pickle.dump(data_list, open(os.path.join(processed_data_folder_path,
                                                    data_binary_name+str(save_part_index)), "wb"))
    
    return data_count

In [ ]:
# total data is around 450,000, of total size about 6GB
# convert_to_binary_sentence_data(sentence_folder_path,processed_data_folder_path,verbose=1,
#                                 data_dump_limit=50,data_count_limit = 210)

convert_to_binary_sentence_data(sentence_folder_path,processed_data_folder_path,verbose=1)

In [ ]:
processed_data = pickle.load(open("processed_data/data0.p", "rb"))
data_df = pandas.DataFrame(processed_data,columns=["year","judge_name","sentence"])
data_df